# --- Day 10: Factory ---


In [ ]:
# --- Support Functions ---
# This section contains support functions used by the main code.

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()


In [ ]:
# --- Part ONE ---

input = read_input('input.txt')

result = 0


print("Part ONE:", result)

In [ ]:
# --- Part TWO ---
input = read_input('input.txt')

result = 0



print("Part TWO:", result)